## Dicas de Jupyter Notebook

Este programa que estamos usando no navegador se chama Jupyter Notebook e é um interpretador de Python interativo. Podemos digitar um comando de Python, apertar _Shift + Enter_ e esse comando é executado, tendo seu resultado impresso na tela.

Existem alguns atalhos bastante úteis para trabalhar com Jupyter Notebook:
    0. 'Esc' - Saí do modo edição da célula atual (modo navegação)
    1. 'Enter' - Edita a célula selecionada (modo navegação)
    2. 'Shift + Enter' - Executa a célula selecionada (modo edição)
    3. 'A' - Adiciona uma nova célula acima da célula atual (modo navegação)
    4. 'B' - Adiciona uma nova célula abaixo da célula atual (modo navegação)
    5. 'X' - Recorta a célula selecionada (modo navegação)
    6. 'Z' - Desfaz a deleção de uma célula (modo navegação)
    7. 'Ctrl + Z' - Desfaz as últimas edições (modo edição)
    8. 'Ctrl + Shift + Z' - Refaz o último comando de desfazer (modo edição)
    9. 'H' - Abre um arquivo de ajuda com todos os atalhos (modo navegação)
    10. 'Tab' - Faz sugestões para completar o código (modo edição)
    11. 'Shift + Tab' - Faz sugestões de assinatura de um método ou função (modo edição)
    12. '?função' - Mostra a documentação de uma dada função (modo edição)
    


In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
train['count'].head()

In [ ]:
train['count'] = np.log(train['count'])

In [ ]:
train['count'].head()

In [ ]:
import math 

In [ ]:
math.log(1000) - math.log(1100) 

In [ ]:
math.log(10) - math.log(11) 

Olhando o tamanho do train e test:

In [ ]:
train.shape, test.shape

In [ ]:
df = train.append(test)

In [ ]:
df.dtypes

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

In [ ]:
df.head()

In [ ]:
df.sort_values('datetime', inplace=True)

In [ ]:
df['rolling_temp'] = df['temp'].rolling(4, min_periods=1).mean()

In [ ]:
df[['temp', 'rolling_temp']].head()

# One Hot Encoding - Dummificar Variáveis

In [ ]:
df.nunique()

In [ ]:
pd.get_dummies(df['dayofweek'], prefix='prefixo').head()

In [ ]:
df.loc[:, (df.nunique() > 2 ) & (df.nunique() < 8)].head()

In [ ]:
for col in df.loc[:, (df.nunique() > 2 ) & (df.nunique() < 8)].columns:
    df = pd.concat([df, pd.get_dummies(df[col], prefix=col).iloc[:, :-1]], 
                   axis =1)
    del df[col]    

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
train_raw = df[~df['count'].isnull()]

In [ ]:
test = df[df['count'].isnull()]

In [ ]:
train_raw.shape, test.shape

In [ ]:
df.columns

**Separando em treino e validação**



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train,valid = train_test_split(train_raw,random_state =42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


In [ ]:
models ={ 'RandomForest': RandomForestRegressor(random_state=42),
        'ExtraTrees': ExtraTreesRegressor(random_state=42),
        'GBM': GradientBoostingRegressor(random_state=42),
        'DecisionTree': DecisionTreeRegressor(random_state=42),
        'AdaBoost': AdaBoostRegressor(random_state=42),
        'KNN1': KNeighborsRegressor(n_neighbors=1),
        'KNN3': KNeighborsRegressor(n_neighbors=3),
        'KNN11':KNeighborsRegressor(n_neighbors=11),
        'SRV': SVR(),
        'Linear Regression': LinearRegression()}

In [ ]:
removed_cols = ['count','casual','registered','datetime']

In [ ]:
feats =[c for c in df.columns if c not in removed_cols]

In [ ]:
from sklearn.metrics import  mean_squared_error

In [ ]:
#rodar modelo fit (aprende com os dados) e o predict prever
def run_model(model,train,valid,feats,y_name):
    model.fit(train[feats],train[y_name])
    preds =model.predict(valid[feats])
    return mean_squared_error(valid[y_name],preds)**(1/2)

In [ ]:
import warnings
warnings.filterwarnings('ignore', category= FutureWarning)

In [ ]:
scores=[]
for name ,model in models.items():
    score=run_model(model,train,valid,feats,'count')
    scores.append(score)
    print (name+':',score)